In [272]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime

#from secret import *

### FUNCTION TO CREATE URL

In [258]:
def find_url(row):
    Base_Link_01= "https://en.wikipedia.org/wiki/"
    Base_Link_02= "_AFL_season"
    Variable= row
    link = Base_Link_01 + Variable + Base_Link_02
    return (link)

### FUNCTION TO CREATE STRIPTIME + DATETIME (WITH DAY)

In [284]:
def structured_times (x):

    struct_time = time.strptime(x, "%Y %d %B (%I:%M %p)")
    dt = datetime.fromtimestamp(time.mktime(struct_time))
    return (dt)

### FUNCTION TO CONVERT TO CONSISTENT DATE SYNTAX

In [285]:
def date_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(":")
    if count >1:
        first_colon_index = x.index(":")
        date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)

### STRIP DAY FOR STRPTIME CONSISTENCY

In [286]:
def day_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[1].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)
    
    


one = "20 September (4.45pm)"
two = "Friday, 14 March (7:50 pm)"
result = day_conversion (two)
print (result)

### FUNCTION TO SORT DATA INTO DICTIONARY FOR EACH MATCH

In [301]:
def match_function(match, this_round, this_year):

    try:

        namestocheck_good = ["Round", "Qualifying ", "Elimination", "Semi", "Preliminary", "Grand"]
        namestocheck_bad = ["NAB"]
        if any (name in this_round for name in namestocheck_good):
            ## but name does not contain this
            #print (this_round)
            test = match.find_all('td')

            time_date= date_conversion(test[0].string)

            
           

            convert_time = day_conversion(time_date)
            convert_time_year = (str(this_year)+ " " + convert_time)
            print (convert_time_year)
            struct_time = structured_times(convert_time_year)
            

            #time_date = test[0].string
            day = time_date.split(',')[0]
            date = time_date.split(',')[1].split('(')[0].strip().split(' ')[0]
            month = time_date.split(',')[1].split('(')[0].strip().split(' ')[1]
            time = (time_date.split('(')[1])[:-1]

            


            #print (day)
            #print (date)
            #print (month)
            #print (year_)
            #print (time)
            #print (time_date)

            home_team = test[1].find('a').string
            home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
            home_goals= home_score.split('.')[0]
            home_behinds= home_score.split('.')[1].split('(')[0]
            home_total = (home_score.split('.')[1].split('(')[1])[:-1]
            #print (home_team)
            #print (home_goals)
            #print (home_behinds)
            #print (home_total)



            away_team = test[3].find('a').string
            away_score = (''.join(text for text in test[3].find_all(text=True) if text.parent.name != "a")).strip()
            away_goals= away_score.split('.')[0]
            away_behinds= away_score.split('.')[1].split('(')[0]
            away_total = (away_score.split('.')[1].split('(')[1])[:-1]
            #print (away_team)
            #print (away_goals)
            #print (away_behinds)
            #print (away_total)



            venue = test[4].find('a').string
            venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
            venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
            final_crowd = venue_crowd_.replace(",", "")
            #print (venue)
            #print (final_crowd)


            match_report_url = test[5].find('a')['href']
            #print (match_report_url)


            ind_match_dict= {
                 "Day": day,
                 "Date": date,
                 "Month": month,
                 "Year": this_year,
                 "Round": this_round,         
                 "Time": time,
                 "Structured Data": struct_time,
                 "Home Team": home_team,
                 "Home Goals": home_goals,
                 "Home Behinds": home_behinds,
                 "Home Total": home_total,
                 "Away Team": away_team,
                 "Away Goals": away_goals,
                 "Away Behinds": away_behinds,
                 "Away Total": away_total,
                 "Venue": venue,
                 "Crowd": final_crowd,
                 "Away Behinds": away_behinds,
                 "Match Link": match_report_url}




            return (ind_match_dict)
        
        else:
            print ("no name in scope", this_round)
    
    except Exception as e:
        print("this row does not have match data", e)
        #print ("/t",convert_time_year)
        #return False



### FUNCTION TO RETURN A LIST OF DATA FOR EACH ROUND

In [288]:
def round_function(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')
    #print (games)
    ## Find the round header
    round_head = (''.join(text for text in games[1].find_all(text=True) if text.parent.name != "a")).strip()
    #print (round_head)
    #if isinstance(round_head,str) == False:
    ## Check is string is empty- empty strings are falsy- so we are asking if not false- double negative- not false = true
    if not round_head:
        round_head = games[1].find('a').string

    ## Only look at matchday tables in each round
    round_matchday = games[2:-1]
    this_round = []
    for match in round_matchday:

        result= match_function(match, round_head, current_year)
        if result:
            this_round.append(result)
    return (this_round)

### RETURN EACH ROUND FOR EACH YEAR FROM FUNCTIONS INTO MEGA LIST

In [302]:
mega_list = []

test_year = 2011

for year in list(range(test_year,test_year + 1)):
#for year in list(range(2009,2018,1)):
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    year_data = soup.find("div", {"id": "mw-content-text"}).find_all('table', {"style": "width: 100%; background-color: #f1f5fc; border: 2px solid #D0E5F5"})
    round_range = len(year_data)
    
    ##years = []
    
    print (current_year)
    for current_round in range(round_range):

        this_round = round_function(current_round, year_data, current_year)
        mega_list.extend(this_round)
        #print (this_round)

    print (current_year)
print (mega_list)


2011
2011 11 March 7:45 pm
this row does not have match data time data '2011 11 March 7:45\xa0pm' does not match format '%Y %d %B (%I:%M %p)'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
2011 24 March (7:40 pm)
2011 25 March (7:40 pm)
2011 26 March (2:10 pm)
2011 26 March (7:10 pm)
2011 26 March (7:10 pm)
2011 27 March (1:10 pm)
2011 27 March (2:10 pm)
2011 27 March (4:10 pm)
this row does not have match data 'NoneType' object has no attribute 'count'
2011 1 April (7:40 pm)
2011 2 April (2:10 pm)
2011

In [104]:
print (mega_list)

[{'Day': 'Friday', 'Date': '14', 'Month': 'March', 'Year': '2014', 'Round': 'Round 1', 'Time': '7:50\xa0pm', 'Structured Data': datetime.datetime(2014, 3, 14, 19, 50), 'Home Team': 'Collingwood', 'Home Goals': '5', 'Home Behinds': '16 ', 'Home Total': '46', 'Away Team': 'Fremantle', 'Away Goals': '17', 'Away Behinds': '14 ', 'Away Total': '116', 'Venue': 'Etihad Stadium', 'Crowd': '37571', 'Match Link': 'http://www.afl.com.au/match-centre/2014/1/coll-v-fre'}, {'Day': 'Saturday', 'Date': '15', 'Month': 'March', 'Year': '2014', 'Round': 'Round 1', 'Time': '4:40\xa0pm', 'Structured Data': datetime.datetime(2014, 3, 15, 16, 40), 'Home Team': 'Greater Western Sydney', 'Home Goals': '15', 'Home Behinds': '9 ', 'Home Total': '99', 'Away Team': 'Sydney', 'Away Goals': '9', 'Away Behinds': '13 ', 'Away Total': '67', 'Venue': 'Spotless Stadium', 'Crowd': '17102', 'Match Link': 'http://www.afl.com.au/match-centre/2014/1/gws-v-syd'}, {'Day': 'Saturday', 'Date': '15', 'Month': 'March', 'Year': '201

### TURN MEGALIST INTO DATAFRAME

In [95]:
df = pd.DataFrame(mega_list)
df.head(400)
#df.shape

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year
0,15,15,Richmond,105,83493,2,Thursday,12,11,Carlton,78,http://afl.com.au/match-centre/2015/1/carl-v-rich,April,Round 1 (Opening Round),2015-04-02 19:20:00,7:20 pm,MCG,2015
1,11,13,Gold Coast,89,27013,4,Saturday,13,17,Melbourne,115,http://www.afl.com.au/match-centre/2015/1/melb...,April,Round 1 (Opening Round),2015-04-04 13:40:00,1:40 pm,MCG,2015
2,6,9,Essendon,60,23274,4,Saturday,12,10,Sydney,72,http://afl.com.au/match-centre/2015/1/syd-v-ess,April,Round 1 (Opening Round),2015-04-04 16:35:00,4:35 pm,ANZ Stadium,2015
3,14,12,Collingwood,86,31240,4,Saturday,8,11,Brisbane Lions,74,http://www.afl.com.au/match-centre/2015/1/bl-v...,April,Round 1 (Opening Round),2015-04-04 18:20:00,6:20 pm,Gabba,2015
4,3,14,West Coast,87,22366,4,Saturday,13,14,Western Bulldogs,97,http://www.afl.com.au/match-centre/2015/1/wb-v...,April,Round 1 (Opening Round),2015-04-04 19:20:00,7:20 pm,Etihad Stadium,2015
5,15,12,Greater Western Sydney,87,18794,5,Sunday,12,11,St Kilda,78,http://www.afl.com.au/match-centre/2015/1/stk-...,April,Round 1 (Opening Round),2015-04-05 13:10:00,1:10 pm,Etihad Stadium,2015
6,9,9,North Melbourne,63,45607,5,Sunday,14,21,Adelaide,140,http://www.afl.com.au/match-centre/2015/1/adel...,April,Round 1 (Opening Round),2015-04-05 14:50:00,2:50 pm,Adelaide Oval,2015
7,8,10,Port Adelaide,68,34099,5,Sunday,9,11,Fremantle,75,http://www.afl.com.au/match-centre/2015/1/fre-...,April,Round 1 (Opening Round),2015-04-05 16:40:00,4:40 pm,Domain Stadium,2015
8,13,8,Geelong,61,73584,6,Monday,21,17,Hawthorn,123,http://www.afl.com.au/match-centre/2015/1/haw-...,April,Round 1 (Opening Round),2015-04-06 15:20:00,3:20 pm,MCG,2015
9,8,9,Carlton,62,34588,10,Friday,11,20,West Coast,131,http://www.afl.com.au/match-centre/2015/2/wce-...,April,Round 2,2015-04-10 18:10:00,6:10 pm,Domain Stadium,2015
